# 1. Imports of necessary libraries

In [77]:
import keras #High level interface of TensorFlow
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical 
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
from keras.optimizers import RMSprop
import tensorflowjs as tfjs
import pandas as pd
from sklearn.model_selection import train_test_split

# 2.Data importing& preprocessing

## 2.1 Importing and entry processing

In [78]:
train_set = pd.read_csv("../data/train.csv") # raw data

# DV
Y_train = train_set["label"]
# IV
X_train = train_set.drop(labels = ["label"], axis = 1)
#X_train

## 2.2Normalization to [0-1]  range

In [79]:
X_train = X_train / 255.0


X_train = X_train.values.reshape(-1,28,28,1)

Y_train = to_categorical(Y_train, num_classes = 10)

## 2.3Data spliting into train & valid set 

In [80]:
X_train, X_valid , Y_train , Y_valid = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42)

## 2.4 Augmentation

In [81]:
igd = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range = 0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

igd.fit(X_train)

# 3. Building the architecture of Keras model & Compilation

## 3.1 Architecture

In [85]:
model = Sequential(
        [
            Conv2D(32, (5,5),activation = "relu",padding = 'Same',input_shape = (28,28,1)),
            Conv2D(32, (5,5),activation = "relu",padding = 'Same'),
            MaxPool2D(pool_size = (2, 2)),
            Dropout(0.25),
            Conv2D(64, (3,3), activation = "relu", padding ='Same'),
            Conv2D(64, (3,3),activation = "relu", padding = 'Same'),
            MaxPool2D(pool_size = (2, 2), strides = (2,2)),
            Dropout(0.25),
            Flatten(),
            Dense(units = 256, activation = "relu"),
            Dense(units = 10, activation = "softmax") ])

'# Create model using Keras Sequential API\nmodel = Sequential()\n\n\n# CONVOLUTIONAL/MAXPOOL LAYERS\n# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = \n\n# Convolutional 2D layer #1\n# Computes 32 features using a 5x5 filter with ReLU activation.\nmodel.add(Conv2D(\n    filters = 32, \n    kernel_size = (5,5),\n    padding = \'Same\', \n    activation =\'relu\',\n    input_shape = (28,28,1)\n))\n\n# Convolutional 2D layer #2\n# Computes 32 features using a 5x5 filter with ReLU activation.\nmodel.add(Conv2D(\n    filters = 32, \n    kernel_size = (5,5),\n    padding = \'Same\', \n    activation =\'relu\'\n))\n\n# Pooling layer #1\n# Max pooling layer with a 2x2 filter\nmodel.add(MaxPool2D(pool_size=(2,2)))\n\n# Dopout operation; 0.75 probability that element will be kept\nmodel.add(Dropout(0.25))\n\n# Convolutional 2D layer #3\n# Computes 64 features using a 3x3 filter with ReLU activation.\nmodel.add(Conv2D(\n    filters = 64, \n    kernel_size = (3,3),\n    pad

## 3.2 Compilation

In [86]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

# 4. Training

In [87]:
model.fit_generator(
    igd.flow(X_train, Y_train, batch_size = 128),
    epochs = 30,
    steps_per_epoch = X_train.shape[0] // 128,
    validation_data = (X_valid, Y_valid)
    )

Epoch 1/30
262/262 [==============================] - 276s 1s/step - loss: 0.4067 - acc: 0.8708 - val_loss: 0.0626 - val_acc: 0.9811
Epoch 2/30
262/262 [==============================] - 268s 1s/step - loss: 0.1135 - acc: 0.9652 - val_loss: 0.0475 - val_acc: 0.9856
Epoch 3/30
262/262 [==============================] - 257s 980ms/step - loss: 0.0820 - acc: 0.9737 - val_loss: 0.0405 - val_acc: 0.9874
Epoch 4/30
262/262 [==============================] - 259s 987ms/step - loss: 0.0677 - acc: 0.9789 - val_loss: 0.0377 - val_acc: 0.9886
Epoch 5/30
262/262 [==============================] - 259s 989ms/step - loss: 0.0614 - acc: 0.9806 - val_loss: 0.0410 - val_acc: 0.9862
Epoch 6/30
262/262 [==============================] - 261s 998ms/step - loss: 0.0503 - acc: 0.9842 - val_loss: 0.0318 - val_acc: 0.9907
Epoch 7/30
262/262 [==============================] - 262s 999ms/step - loss: 0.0456 - acc: 0.9860 - val_loss: 0.0281 - val_acc: 0.9912
Epoch 8/30
262/262 [==============================] - 

# Saving a model

In [88]:
tfjs.converters.save_keras_model(model, '../model')